## 6.5 RNN

6.2节的图像分类用了CNN， 本节介绍RNN的PyTorch实现。

相关资料：
- 可以参考我的博客：https://blog.csdn.net/sinat_33761963/article/details/53521185
- 我的另一篇博客：https://blog.csdn.net/sinat_33761963/article/details/80195636
- 网上的课程

很多复杂的模型都是基于RNN的结构，因此本文先介绍最基础的RNN，为以后的学习做好准备。

### 6.5.1准备数据
为了简化，这里先捏造一批数据，注意打印出来看看最终z,y的数据形式，大小为（1，1，100），第一个维度是batch_size

- unsqueeze()这个方法关注下，会经常用到

In [7]:
import numpy as np
import torch

steps = np.linspace(0, np.pi*2, 10, dtype=np.float32)
x = np.sin(steps)
y = np.cos(steps)

x = torch.from_numpy(x).unsqueeze(0).unsqueeze(-1)
y = torch.from_numpy(y).unsqueeze(0).unsqueeze(-1)
print(x[:10])

tensor([[[ 0.0000],
         [ 0.6428],
         [ 0.9848],
         [ 0.8660],
         [ 0.3420],
         [-0.3420],
         [-0.8660],
         [-0.9848],
         [-0.6428],
         [ 0.0000]]])


### 6.5.2 构建LSTM网路结构

lstm是RNN的其中一中结构，也是目前业界非常常用的一种结构。

- nn.LSTM可以直接创建一个lstm结构，其中参数input_size是输入向量的维度， hidden_size是输出隐向量的维度，batch_first=True表示输入与输出的第一维是batch_size。
- lstm中每步都有三个输入输出，输入：当前步的输入x， 上一步的输出c和h；输出：隐向量h, 这一步的c和h,这样就可以理解forward中的第一行代码的参数。
- def init_hidden是供外部调用的一个参数初始化的函数。

In [2]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, INPUT_SIZE):
        super(LSTM, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=INPUT_SIZE,
            hidden_size=20,
            batch_first=True
        )
        
        self.out = nn.Linear(20, 1)
    
    def forward(self, x, h_state, c_state):
        r_out, (h_state, c_state) = self.lstm(x, (h_state, c_state))
        outputs = self.out(r_out[0,:]).unsqueeze(0)
        return outputs, h_state, c_state
    
    def init_hidden(self):
        h_state = torch.randn(1,1,20)
        c_state = torch.randn(1,1,20)
        return h_state, c_state

### 6.5.3 训练


In [11]:
lstm = LSTM(INPUT_SIZE=1)
optimizer = torch.optim.SGD(lstm.parameters(), lr=0.001)
loss_func = nn.MSELoss()

h_state, c_state = lstm.init_hidden()

for step in range(100):
    prediction, h_state, c_state = lstm(x, h_state, c_state)
    h_state = h_state.data
    c_state = c_state.data
    
    loss = loss_func(prediction, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if step % 10 == 0:
        print("loss:%.3f" % (loss.data))
    

loss:0.546
loss:0.536
loss:0.536
loss:0.536
loss:0.536
loss:0.536
loss:0.535
loss:0.535
loss:0.535
loss:0.535


### 总结

1.事先学习RNN家族的理论知识
2.明白如何构建RNN，Pytorch提供了RNN,LSTM,GRU三种封装好的结构接口，也提供了RNNCell, LSTMCell, GRUCell三种循环网络的单元结构，官网连接：https://pytorch.org/docs/stable/nn.html#recurrent-layers
3.做循环神经网络，输入输出参数的维度，数据在整个过程中的形态都需要搞清楚。